In [ ]:
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
import seaborn as sns

# Style général pour des graphismes propres
sns.set(style="whitegrid", palette="muted", font_scale=1.2)
import matplotlib

matplotlib.rcParams['font.family'] = 'DejaVu Sans'  # Police compatible LaTeX


def flip_random_spin(spin_array, generator=None):
    """
    Sélectionne un spin aléatoire, l'inverse et calcule la variation d'énergie ΔE
    uniquement liée au spin retourné (pas besoin de recalculer tout le système)
    """
    N = spin_array.size
    if generator is None:
        generator = rnd.default_rng()
    flip_index = generator.integers(0, N)

    # Cas des bords
    if flip_index == 0:
        delta_energy = -2 * spin_array[flip_index] * spin_array[flip_index + 1]
    elif flip_index == N - 1:
        delta_energy = -2 * spin_array[flip_index] * spin_array[flip_index - 1]
    else:
        delta_energy = -2 * spin_array[flip_index] * (spin_array[flip_index - 1] + spin_array[flip_index + 1])

    new_array = spin_array.copy()
    new_array[flip_index] *= -1  # Flip du spin
    return new_array, delta_energy, flip_index


def metropolis_step(spin_array, current_energy, current_magnet, temperature, generator=None):
    """
    Étape Metropolis : propose un flip, calcule ΔE et ΔM, accepte ou rejette
    selon la probabilité de Boltzmann.
    """
    if generator is None:
        generator = rnd.default_rng()
    new_array, delta_E, flip_idx = flip_random_spin(spin_array, generator)
    delta_M = 2 * new_array[flip_idx]
    accepted = False

    # Critère d'acceptation de Metropolis
    if delta_E < 0 or generator.random() < np.exp(-delta_E / temperature):
        spin_array = new_array
        current_energy += delta_E
        current_magnet += delta_M
        accepted = True

    return spin_array, current_energy, current_magnet, accepted


def compute_specific_heat(energy_samples, T):
    """
    Capacité calorifique selon l’énoncé :
    Cv = (<E²> - <E>²) / T²
    """
    av_E = np.mean(energy_samples)
    return (np.mean(energy_samples ** 2) - av_E ** 2) / T ** 2


def ising_simulation(N, T, SAMPLE_STEPS, generator, init_type='aligned'):
    """
    Simulation complète :
    - Initialisation (tous spins alignés ou aléatoires)
    - Évolution par l'algorithme de Metropolis
    - Retourne les énergies et magnétisations à chaque étape
    """
    if init_type == 'aligned':
        spins = np.ones(N, dtype=np.byte)
    else:
        spins = generator.choice([-1, 1], size=N).astype(np.byte)

    energy = -np.sum(spins[1:] * spins[:-1])
    magnet = np.sum(spins)

    all_E = np.zeros(SAMPLE_STEPS)
    all_M = np.zeros(SAMPLE_STEPS)
    all_E[0] = energy
    all_M[0] = magnet

    for i in range(1, SAMPLE_STEPS):
        spins, energy, magnet, _ = metropolis_step(spins, energy, magnet, T, generator)
        all_E[i] = energy
        all_M[i] = magnet

    return all_E, all_M


N = 1000  # Nombre de spins
SAMPLE_STEPS = 10000  # Nombre d'étapes Monte Carlo
generator = rnd.default_rng()
temperatures_study = np.linspace(0.1, 5.0, 15)  # Pour étude complète en T

for T in [0.1, 1.0, 20.0]:
    # Type d'initialisation : aligné pour T faible, aléatoire pour T élevé
    init_type = 'aligned' if T <= 1.0 else 'random'
    all_E, all_M = ising_simulation(N, T, SAMPLE_STEPS, generator, init_type)

    # --- Graphique Énergie et Magnétisation ---
    plt.figure(figsize=(10, 5))
    plt.plot(all_E, label=r'$\mathrm{Energie}$', color='dodgerblue', alpha=0.7)
    plt.plot(all_M, label=r'$\mathrm{Magnetisation}$', color='crimson', alpha=0.7)
    plt.title(f"Énergie et Magnétisation vs Étapes - T = {T}")
    plt.xlabel("Étape")
    plt.ylabel("Valeur")
    plt.legend()
    plt.tight_layout()
    plt.show()

    # --- Histogrammes normalisés ---
    # stat="density" normalise l'axe vertical pour que l'aire sous la courbe soit 1
    fig, ax = plt.subplots(2, 1, figsize=(8, 6))
    sns.histplot(all_E, bins=50, stat="density", kde=True, color='dodgerblue', ax=ax[0])
    ax[0].set_title("Distribution de l'Énergie")
    ax[0].set_xlabel("Énergie")
    ax[0].set_ylabel("Densité")

    sns.histplot(all_M, bins=50, stat="density", kde=True, color='crimson', ax=ax[1])
    ax[1].set_title("Distribution de la Magnétisation")
    ax[1].set_xlabel("Magnétisation")
    ax[1].set_ylabel("Densité")

    plt.tight_layout()
    plt.show()

    # --- Calcul des observables ---
    avg_E = np.mean(all_E)
    avg_M = np.mean(np.abs(all_M))
    Cv = compute_specific_heat(all_E, T)

    print(
        f"\nT = {T:.1f} → Énergie moyenne: {avg_E:.2f}, Magnétisation moyenne: {avg_M:.2f}, Cv = ({Cv:.2f}) selon Cv = (<E²> - <E>²)/T²")

avg_E_list = []
avg_M_list = []
Cv_list = []

for T in temperatures_study:
    init_type = 'aligned' if T <= 1.0 else 'random'
    all_E, all_M = ising_simulation(N, T, SAMPLE_STEPS // 2, generator, init_type)
    avg_E_list.append(np.mean(all_E))
    avg_M_list.append(np.mean(np.abs(all_M)))
    Cv_list.append(compute_specific_heat(all_E, T))

# --- Graphiques finaux ---
fig, ax = plt.subplots(3, 1, figsize=(10, 12))
ax[0].plot(temperatures_study, avg_E_list, marker='o', color='dodgerblue', label=r'$\langle E \rangle$')
ax[0].set_ylabel("Énergie moyenne")
ax[0].set_title(r"Énergie, Magnétisation et $C_v$ en fonction de la température")
ax[0].legend()

ax[1].plot(temperatures_study, avg_M_list, marker='o', color='crimson', label=r'$\langle |M| \rangle$')
ax[1].set_ylabel("Magnétisation moyenne")
ax[1].legend()

ax[2].plot(temperatures_study, Cv_list, marker='o', color='green', label=r'$C_v$')
ax[2].set_xlabel("Température")
ax[2].set_ylabel("Capacité calorifique")
ax[2].legend()

plt.tight_layout()
plt.show()

